In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/hateful-memes

Mounted at /content/drive
/content/drive/MyDrive/hateful-memes


In [6]:
!ls

 ClipTrain.ipynb   dev.csv	     'OCR, Inpainting and Speech Tagging'   train.csv
 cyw_features	   dev_features.csv   README.md				    train_features.csv
 data		   lightning_logs     test.csv
 data_inpainted    main.py	      test_features.csv


In [26]:
# Integrate Fairface Labels

# load files
df = pd.read_csv('test.csv')
face_features = pd.read_csv('cyw_features/race_gender_age_all_10k.csv')

# make sure all original text ends with ending punctuation: . ! ?
target_ids = set()
for index, row in df.iterrows():
    target_ids.add(row['img'].split('/')[1][:5])
    if row['text'][-1].isalpha():
        df.loc[index, 'text'] += "."

# cancatenate race features to text
for index, row in face_features.iterrows():
    id = row['face_name_align'].split('/')[1][:5]
    if id in target_ids:
        face_text = " " + row['race'] + " " + row['gender'] + ", " + row['age'] + " years old."
        idx = df[df['id'] == int(id)].index
        df.loc[idx,'text'] += face_text

# verify and save
# assert(df.loc[0, 'text'][-10:] == "years old.")
df.to_csv('cyw_features/test_w_face.csv')

In [28]:
df.loc[0, 'text']

'handjobs sold seperately. White Female, 20-29 years old.'

In [31]:
# Integrate Entities Labels

# include fairface labels?
include_face = True
if include_face:
    load_path = 'cyw_features/test_w_face.csv'
    save_path = 'cyw_features/test_w_face_n_entities.csv'
else:
    load_path = 'test.csv'
    save_path = 'cyw_features/test_w_entities.csv'

# load files
df = pd.read_csv(load_path)
entities_features = pd.read_csv('cyw_features/entities_all_10k.csv')

# make sure all original text ends with ending punctuation: . ! ?
target_ids = set()
for index, row in df.iterrows():
    target_ids.add(row['img'].split('/')[1][:5])
    if row['text'][-1].isalpha():
        df.loc[index, 'text'] += "."

# cancatenate race features to text
score_cols = entities_features.columns[1:]
for index, row in entities_features.iterrows():
    id = row['filename'][:5]
    if id in target_ids:
        entities_text = ''
        for i, c in enumerate(score_cols):
            if i % 2 == 1 and type(row[c]) == str: # valid description col
                entities_text += " "
                entities_text += row[c]

        idx = df[df['id'] == int(id)].index
        df.loc[idx,'text'] += entities_text

# verify and save
# assert(df.loc[0, 'text'][-50:] == "Facial hair Forehead Hair Meter Face Pressure head")

df.to_csv(save_path)

In [32]:
df.loc[0, 'text']

'handjobs sold seperately. White Female, 20-29 years old. Blanchard Ryan Open Water Internet meme Image Handjob video recording Actor Subtitle Getty Images'